### Stacking the best regression predictor and the neural network

In [139]:
import importlib
import helper_functions
import pandas as pd
importlib.reload(helper_functions)
from helper_functions import *
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline, FeatureUnion, _transform_one
from sklearn.externals.joblib import Parallel, delayed

In [140]:
# Defining pipeline as in other examples
trans_pipeline = Pipeline([
    ('impute_numerical', DFTransform(lambda X: fill_numerical_nans(X))),
    ('impute_categorical', DFTransform(lambda X: impute_categorical(X))),
    ('impute_special_cases', DFTransform(lambda X: impute_special_cases(X))),
    ('drop_features', DFTransform(lambda X: drop_features(X))),
    ('ordinal_features', DFTransform(lambda X: encode_ordinals(X))),
    ('check_nans', DFTransform(lambda X: check_nans(X))),
    ('encode_dummies', DFTransform(lambda X: create_dummies(X)))
    ])

In [141]:
# Load the data
train_df =  pd.read_csv('data/train.csv')
X_train = train_df.drop(['SalePrice','Id'], axis=1)
y_train = train_df['SalePrice']
X_test = pd.read_csv('data/test.csv').drop(['Id'], axis=1)
X_train, y_train = prepare_inputs(X_train, y_train)

# Transforming the input
X_combined = pd.concat((X_train, X_test)).reset_index(drop=True) 
X_tranformed = trans_pipeline.fit_transform(X_combined)

# Split the transformed input back
X_train_trans = X_tranformed[:X_train.shape[0]] 
X_test_trans = X_tranformed[X_train.shape[0]:]

Creating dummies...
Starting with input of shape: (2915, 78)
Returning output of shape: (2915, 219)


In [142]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train_trans, y_train, test_size=.2, random_state=42)

# Split the training set into two subsets for prediction and blending
X_train_subset1, X_train_subset2, y_train_subset1, y_train_subset2 = \
                                    train_test_split(X_train, y_train, test_size=.3, random_state=42)

In [143]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Ridge

# Define first estimator
adaboost_estimator = AdaBoostRegressor(base_estimator=Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=42, solver='cholesky', tol=0.001),
         learning_rate=.001, loss='linear', n_estimators=100,
         random_state=42)

In [144]:
# Train the first estimator on the first subset
adaboost_estimator.fit(X_train_subset1, y_train_subset1)

AdaBoostRegressor(base_estimator=Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=42, solver='cholesky', tol=0.001),
         learning_rate=0.001, loss='linear', n_estimators=100,
         random_state=42)

In [145]:
# Predict with the first estimator on the second subset
prediction_adaboost = adaboost_estimator.predict(X_train_subset2)

In [146]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint  
import livelossplot as lp
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation


# Define the build function for the second estimator
def build_nn():
    model = Sequential()
    model.add(Dense(219, kernel_initializer='normal', activation='relu', input_shape=(219,)))
    model.add(Dropout(.3))
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(.1))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))

    # Compiling the mode
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    return model

In [147]:
# Define pipeline with scaler for second regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp_regressor', KerasRegressor(build_fn=build_nn, batch_size=5, epochs=1000, verbose=1))
])

In [148]:
# Train the second estimator on the first subset
pipeline.fit(X_train_subset1, y_train_subset1)

Epoch 1/1000
814/814 [==============================] - 1s 1ms/step - loss: 26.8119
Epoch 2/1000
814/814 [==============================] - 0s 481us/step - loss: 3.7402
Epoch 3/1000
814/814 [==============================] - 0s 492us/step - loss: 3.2710
Epoch 4/1000
814/814 [==============================] - 0s 537us/step - loss: 2.9910
Epoch 5/1000
814/814 [==============================] - 0s 495us/step - loss: 2.9718
Epoch 6/1000
814/814 [==============================] - 0s 499us/step - loss: 2.8776
Epoch 7/1000
814/814 [==============================] - 1s 693us/step - loss: 3.2406
Epoch 8/1000
814/814 [==============================] - 1s 679us/step - loss: 2.7167
Epoch 9/1000
814/814 [==============================] - 0s 509us/step - loss: 2.9441
Epoch 10/1000
814/814 [==============================] - 0s 517us/step - loss: 2.8646
Epoch 11/1000
814/814 [==============================] - 0s 510us/step - loss: 2.7345
Epoch 12/1000
814/814 [==============================] - 0s 509u

Epoch 190/1000
814/814 [==============================] - 0s 482us/step - loss: 0.2315
Epoch 191/1000
814/814 [==============================] - 0s 453us/step - loss: 0.1977
Epoch 192/1000
814/814 [==============================] - 0s 443us/step - loss: 0.2299
Epoch 193/1000
814/814 [==============================] - 0s 453us/step - loss: 0.2256
Epoch 194/1000
814/814 [==============================] - 0s 454us/step - loss: 0.2187
Epoch 195/1000
814/814 [==============================] - 0s 457us/step - loss: 0.2178
Epoch 196/1000
814/814 [==============================] - 1s 632us/step - loss: 0.1984
Epoch 197/1000
814/814 [==============================] - 0s 544us/step - loss: 0.2213
Epoch 198/1000
814/814 [==============================] - 0s 477us/step - loss: 0.1973
Epoch 199/1000
814/814 [==============================] - 0s 478us/step - loss: 0.2034
Epoch 200/1000
814/814 [==============================] - 0s 501us/step - loss: 0.1862
Epoch 201/1000
814/814 [===================

814/814 [==============================] - 0s 507us/step - loss: 0.1011
Epoch 378/1000
814/814 [==============================] - 0s 471us/step - loss: 0.0849
Epoch 379/1000
814/814 [==============================] - 0s 462us/step - loss: 0.0861
Epoch 380/1000
814/814 [==============================] - 0s 456us/step - loss: 0.0829
Epoch 381/1000
814/814 [==============================] - 0s 467us/step - loss: 0.0842
Epoch 382/1000
814/814 [==============================] - 0s 453us/step - loss: 0.0835
Epoch 383/1000
814/814 [==============================] - 0s 469us/step - loss: 0.0930
Epoch 384/1000
814/814 [==============================] - 0s 464us/step - loss: 0.0788
Epoch 385/1000
814/814 [==============================] - 0s 464us/step - loss: 0.0874
Epoch 386/1000
814/814 [==============================] - 0s 468us/step - loss: 0.0951
Epoch 387/1000
814/814 [==============================] - 0s 470us/step - loss: 0.0851
Epoch 388/1000
814/814 [==============================] - 

814/814 [==============================] - 0s 484us/step - loss: 0.0589
Epoch 472/1000
814/814 [==============================] - 0s 462us/step - loss: 0.0528
Epoch 473/1000
814/814 [==============================] - 0s 442us/step - loss: 0.0577
Epoch 474/1000
814/814 [==============================] - 0s 448us/step - loss: 0.0505
Epoch 475/1000
814/814 [==============================] - 0s 461us/step - loss: 0.0540
Epoch 476/1000
814/814 [==============================] - 0s 450us/step - loss: 0.0493
Epoch 477/1000
814/814 [==============================] - 0s 453us/step - loss: 0.0486
Epoch 478/1000
814/814 [==============================] - 0s 450us/step - loss: 0.0488
Epoch 479/1000
814/814 [==============================] - 0s 464us/step - loss: 0.0486
Epoch 480/1000
814/814 [==============================] - 0s 467us/step - loss: 0.0510
Epoch 481/1000
814/814 [==============================] - 0s 459us/step - loss: 0.0467
Epoch 482/1000
814/814 [==============================] - 

814/814 [==============================] - 0s 491us/step - loss: 0.0148
Epoch 660/1000
814/814 [==============================] - 0s 492us/step - loss: 0.0148
Epoch 661/1000
814/814 [==============================] - 0s 561us/step - loss: 0.0158
Epoch 662/1000
814/814 [==============================] - 0s 580us/step - loss: 0.0171
Epoch 663/1000
814/814 [==============================] - 0s 570us/step - loss: 0.0153
Epoch 664/1000
814/814 [==============================] - 0s 561us/step - loss: 0.0150
Epoch 665/1000
814/814 [==============================] - 0s 572us/step - loss: 0.0140
Epoch 666/1000
814/814 [==============================] - 0s 470us/step - loss: 0.0167
Epoch 667/1000
814/814 [==============================] - 0s 471us/step - loss: 0.0133
Epoch 668/1000
814/814 [==============================] - 0s 465us/step - loss: 0.0130
Epoch 669/1000
814/814 [==============================] - 0s 457us/step - loss: 0.0157
Epoch 670/1000
814/814 [==============================] - 

814/814 [==============================] - 0s 486us/step - loss: 0.0110
Epoch 848/1000
814/814 [==============================] - 0s 459us/step - loss: 0.0092
Epoch 849/1000
814/814 [==============================] - 0s 441us/step - loss: 0.0117
Epoch 850/1000
814/814 [==============================] - 0s 459us/step - loss: 0.0109
Epoch 851/1000
814/814 [==============================] - 0s 438us/step - loss: 0.0121
Epoch 852/1000
814/814 [==============================] - 0s 455us/step - loss: 0.0118
Epoch 853/1000
814/814 [==============================] - 0s 453us/step - loss: 0.0130
Epoch 854/1000
814/814 [==============================] - 0s 456us/step - loss: 0.0102
Epoch 855/1000
814/814 [==============================] - 0s 451us/step - loss: 0.0119
Epoch 856/1000
814/814 [==============================] - 0s 468us/step - loss: 0.0129
Epoch 857/1000
814/814 [==============================] - 0s 451us/step - loss: 0.0126
Epoch 858/1000
814/814 [==============================] - 

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp_regressor', <keras.wrappers.scikit_learn.KerasRegressor object at 0x000001AD21F5F898>)])

In [149]:
# Predict with the second estimator on the second subset
prediction_nn = pipeline.predict(X_train_subset2)

350/350 [==============================] - 0s 626us/step


In [154]:
# Not we define a bagging ensemble for the blending
bagging_estimator = Ridge(alpha=10, random_state=42, solver='cholesky', tol=0.001)

In [155]:
# This blender now we train on the predictions of the first layer
X_blended = np.column_stack((prediction_nn, prediction_adaboost))

bagging_estimator.fit(X_blended, y_train_subset2)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=42, solver='cholesky', tol=0.001)

In [156]:
# And now we use the stack to make a prediction on unseen data
test_prediction_ada = adaboost_estimator.predict(X_test)
test_prediction_nn = pipeline.predict(X_test)

X_test_blended = np.column_stack((test_prediction_nn, test_prediction_ada))
y_predicted = bagging_estimator.predict(X_test_blended)

292/292 [==============================] - 0s 275us/step


In [157]:
print_benchmark(y_test, y_predicted)

R2-score: 0.874058213494
RMSE (log): 0.14103564617996528
